# OSRM

In [ ]:
!pip install polyline geocoder -qq

In [ ]:
import logging
import requests
import json
import polyline
import folium
from folium.plugins import MeasureControl
import geocoder

from functools import lru_cache

logger = logging.getLogger(__name__)
DEBUG = True

@lru_cache(maxsize=None)
def geocode(location):
    return _geocode(location)

def _geocode(location):
    import geocoder
    g = geocoder.osm(location)
    return g.latlng

@lru_cache(maxsize=None)
def get_route(olat, olng, dlat, dlng):
    response = _get_route(olat, olng, dlat, dlng)
    return response

def _get_route(olat, olng, dlat, dlng):
    url = f'http://router.project-osrm.org/route/v1/driving/{olng},{olat};{dlng},{dlat}?alternatives=false&steps=false'
    # logger.debug(url)
    response = None

    try:
        logger.debug(f'====== OSRM: {url}')
        response = requests.get(url, verify=False)
    except Exception as ex:
        raise
    
    # logger.debug(response.text)
    if response and response.text:
        response_dict = json.loads(response.text)
        #possible = pd.DataFrame([{'Distance': (route['distance'] / 1000) *  0.621371 , route['weight_name']: route['weight']} for route in response_dict['routes']])
        return response_dict
    else:
        return None

def get_routing_map(origin, destination):
    orig_latlng = geocode(origin)
    dest_latlng = geocode(destination)

    resp = get_route(orig_latlng[0], orig_latlng[1], dest_latlng[0], dest_latlng[1])

    decoded = polyline.decode(resp["routes"][0]['geometry'])
    distance = resp["routes"][0]['distance'] * 0.000621371
    duration = resp["routes"][0]['duration'] / 60

    map2 = folium.Map(location=(orig_latlng[0], orig_latlng[1]), zoom_start=6,
                                    control_scale=True)
    map2.add_child(MeasureControl(
        primary_length_unit='miles',
        secondary_length_unit='meters',
        primary_area_unit='acres',
        secondary_area_unit='sqmeters')
    )

    folium.PolyLine(locations=decoded, color="blue").add_to(map2)

    print(f"{duration} minutes")
    print(f"{distance} miles")

    return map2

In [ ]:
get_routing_map('Plymouth, MN', 'San Diego, CA')

2086.9633333333336 minutes
1935.3934499907998 miles


In [ ]:
get_routing_map('Ragama, Sri Lanka', 'Kandy, Sri Lanka')

123.43166666666666 minutes
64.9220226849 miles


In [15]:
import geocoder
g = geocoder.arcgis('Redlands, CA')
g.json

{'address': 'Redlands, California',
 'bbox': {'northeast': [34.12838000000007, -117.10958999999995],
  'southwest': [33.98238000000007, -117.25558999999994]},
 'confidence': 2,
 'lat': 34.05538000000007,
 'lng': -117.18258999999995,
 'ok': True,
 'quality': 'Locality',
 'raw': {'extent': {'xmax': -117.10958999999995,
   'xmin': -117.25558999999994,
   'ymax': 34.12838000000007,
   'ymin': 33.98238000000007},
  'feature': {'attributes': {'Addr_Type': 'Locality', 'Score': 100},
   'geometry': {'x': -117.18258999999995, 'y': 34.05538000000007}},
  'name': 'Redlands, California'},
 'score': 100,
 'status': 'OK'}